#### Imports

In [3]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

--2024-09-26 12:30:35--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-09-26 12:30:35--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-09-26 12:30:35--  http://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1
Resolving nchc.

In [2]:
#!pip install numpy==1.26.4 --force
numpy.__version__

'1.26.4'

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib as ta
from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Define Ticker List | BIST30

In [5]:
tickers = [
    "AKBNK.IS", "ALARK.IS", "ASELS.IS", "ASTOR.IS", "BIMAS.IS",
    "BRSAN.IS", "DOAS.IS", "EKGYO.IS", "ENKAI.IS", "EREGL.IS",
    "FROTO.IS", "GARAN.IS", "GUBRF.IS", "HEKTS.IS", "ISISCTR.IS",
    "KCHOL.IS", "KONTR.IS", "KOZAL.IS", "KRDMD.IS", "OYAKC.IS",
    "PETKM.IS", "PGSUS.IS", "SAHOL.IS", "SASA.IS", "SISE.IS",
    "TCELL.IS", "THYAO.IS", "TOASO.IS", "TUPRS.IS", "YKBNK.IS"
]

In [6]:
# Define function to get data and calculate indicators
def get_stock_data(ticker):
    # Download 6 months of historical data with a 15-minute interval
    data = yf.download(ticker, period='2y', interval='1d')

    # Calculate EMAs for Vegas Tunnel
    data['EMA_144'] = ta.EMA(data['Close'], timeperiod=144)
    data['EMA_169'] = ta.EMA(data['Close'], timeperiod=169)

    # Calculate RSI for trend confirmation
    data['RSI'] = ta.RSI(data['Close'], timeperiod=14)

    # Calculate ATR for profit target
    data['ATR'] = ta.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)

    return data

In [7]:
# Define strategy logic for long/short positions
def apply_strategy(data):
    signals = []

    for i in range(len(data)):
        # Check if price is above both EMAs (Long Signal)
        if data['Close'].iloc[i] > data['EMA_144'].iloc[i] and data['Close'].iloc[i] > data['EMA_169'].iloc[i]:
            if data['RSI'].iloc[i] > 50:  # Confirm with RSI > 50
                signals.append(('Long', data.index[i], data['Close'].iloc[i], data['ATR'].iloc[i]))

        # Check if price is below both EMAs (Short Signal)
        elif data['Close'].iloc[i] < data['EMA_144'].iloc[i] and data['Close'].iloc[i] < data['EMA_169'].iloc[i]:
            if data['RSI'].iloc[i] < 50:  # Confirm with RSI < 50
                signals.append(('Short', data.index[i], data['Close'].iloc[i], data['ATR'].iloc[i]))

    return signals

In [8]:
# Set profit target and stop-loss using ATR
def calculate_targets(position, entry_price, atr):
    if position == 'Long':
        profit_target = entry_price + (2 * atr)  # Set a 2x ATR profit target for long
        stop_loss = entry_price - atr  # Set stop-loss at 1 ATR below entry
    else:
        profit_target = entry_price - (2 * atr)  # Set a 2x ATR profit target for short
        stop_loss = entry_price + atr  # Set stop-loss at 1 ATR above entry

    return profit_target, stop_loss

In [9]:
# Main function to loop through tickers and apply strategy
def main():
    for ticker in tickers:
        print(f'Processing {ticker}...')

        # Get stock data
        data = get_stock_data(ticker)

        # Apply the strategy to identify signals
        signals = apply_strategy(data)

        # Print the signals and targets
        for signal in signals:
            position, date, entry_price, atr = signal
            profit_target, stop_loss = calculate_targets(position, entry_price, atr)
            print(f'{position} signal for {ticker} on {date}')
            print(f'Entry Price: {entry_price}, Profit Target: {profit_target}, Stop Loss: {stop_loss}\n')

# Run the main function
if __name__ == '__main__':
    main()

Processing AKBNK.IS...


[*********************100%***********************]  1 of 1 completed


Short signal for AKBNK.IS on 2023-05-29 00:00:00
Entry Price: 14.890000343322754, Profit Target: 13.050965149796909, Stop Loss: 15.809517940085676

Short signal for AKBNK.IS on 2023-05-31 00:00:00
Entry Price: 15.800000190734863, Profit Target: 13.879403414835112, Stop Loss: 16.76029857868474

Long signal for AKBNK.IS on 2023-06-02 00:00:00
Entry Price: 17.549999237060547, Profit Target: 19.523268878439055, Stop Loss: 16.563364416371293

Long signal for AKBNK.IS on 2023-06-05 00:00:00
Entry Price: 18.170000076293945, Profit Target: 20.125179116195643, Stop Loss: 17.1924105563431

Long signal for AKBNK.IS on 2023-06-06 00:00:00
Entry Price: 18.040000915527344, Profit Target: 19.918381256394486, Stop Loss: 17.10081074509377

Long signal for AKBNK.IS on 2023-06-07 00:00:00
Entry Price: 18.389999389648438, Profit Target: 20.26278108024368, Stop Loss: 17.453608544350814

Long signal for AKBNK.IS on 2023-06-08 00:00:00
Entry Price: 18.649999618530273, Profit Target: 20.493296837259617, Stop 

[*********************100%***********************]  1 of 1 completed


Long signal for ALARK.IS on 2023-06-07 00:00:00
Entry Price: 72.94999694824219, Profit Target: 79.26134610266672, Stop Loss: 69.79432237102992

Long signal for ALARK.IS on 2023-06-08 00:00:00
Entry Price: 73.75, Profit Target: 80.03196663600023, Stop Loss: 70.60901668199988

Long signal for ALARK.IS on 2023-06-09 00:00:00
Entry Price: 79.0, Profit Target: 85.66896880116037, Stop Loss: 75.66551559941982

Long signal for ALARK.IS on 2023-06-12 00:00:00
Entry Price: 76.0, Profit Target: 82.74975696191734, Stop Loss: 72.62512151904133

Long signal for ALARK.IS on 2023-06-13 00:00:00
Entry Price: 72.69999694824219, Profit Target: 79.66048533775415, Stop Loss: 69.2197527534862

Long signal for ALARK.IS on 2023-06-15 00:00:00
Entry Price: 72.1500015258789, Profit Target: 79.00827912615286, Stop Loss: 68.72086272574194

Long signal for ALARK.IS on 2023-06-16 00:00:00
Entry Price: 72.0, Profit Target: 78.8041153505055, Stop Loss: 68.59794232474725

Short signal for ALARK.IS on 2023-06-21 00:00:

...

Long signal for TOASO.IS on 2023-05-29 00:00:00
Entry Price: 196.5, Profit Target: 216.80615731117828, Stop Loss: 186.34692134441084

Long signal for TOASO.IS on 2023-05-30 00:00:00
Entry Price: 207.0, Profit Target: 227.5700032175227, Stop Loss: 196.71499839123865

Long signal for TOASO.IS on 2023-05-31 00:00:00
Entry Price: 214.5, Profit Target: 235.12928826601996, Stop Loss: 204.18535586699002

Long signal for TOASO.IS on 2023-06-01 00:00:00
Entry Price: 206.5, Profit Target: 226.8986243824817, Stop Loss: 196.30068780875914

Long signal for TOASO.IS on 2023-06-02 00:00:00
Entry Price: 206.89999389648438, Profit Target: 226.99871740929103, Stop Loss: 196.85063214008105

Long signal for TOASO.IS on 2023-06-05 00:00:00
Entry Price: 215.0, Profit Target: 235.07738742693095, Stop Loss: 204.96130628653452

Long signal for TOASO.IS on 2023-06-06 00:00:00
Entry Price: 216.39999389648438, Profit Target: 236.28613892838345, Stop Loss: 206.45692138053485

Long signal for TOASO.IS on 2023-06-07

[*********************100%***********************]  1 of 1 completed


Short signal for TUPRS.IS on 2023-05-29 00:00:00
Entry Price: 68.19999694824219, Profit Target: 61.806842139898606, Stop Loss: 71.39657435241398

Short signal for TUPRS.IS on 2023-05-31 00:00:00
Entry Price: 67.0, Profit Target: 60.69110527001103, Stop Loss: 70.15444736499448

Short signal for TUPRS.IS on 2023-06-01 00:00:00
Entry Price: 67.6500015258789, Profit Target: 61.54174213374629, Stop Loss: 70.70413122194522

Long signal for TUPRS.IS on 2023-06-05 00:00:00
Entry Price: 73.0999984741211, Profit Target: 79.35609949784211, Stop Loss: 69.97194796226059

Long signal for TUPRS.IS on 2023-06-06 00:00:00
Entry Price: 73.30000305175781, Profit Target: 79.45209707748145, Stop Loss: 70.22395603889599

Long signal for TUPRS.IS on 2023-06-07 00:00:00
Entry Price: 77.44999694824219, Profit Target: 83.89122689700287, Stop Loss: 74.22938197386185

Long signal for TUPRS.IS on 2023-06-08 00:00:00
Entry Price: 77.6500015258789, Profit Target: 84.05971504972811, Stop Loss: 74.4451447639543

Long 

[*********************100%***********************]  1 of 1 completed


Short signal for YKBNK.IS on 2023-05-29 00:00:00
Entry Price: 9.34000015258789, Profit Target: 8.332442222811057, Stop Loss: 9.843779117476307

Long signal for YKBNK.IS on 2023-05-30 00:00:00
Entry Price: 10.270000457763672, Profit Target: 11.3384471504387, Stop Loss: 9.735777111426158

Long signal for YKBNK.IS on 2023-05-31 00:00:00
Entry Price: 10.199999809265137, Profit Target: 11.292128853786968, Stop Loss: 9.653935287004222

Long signal for YKBNK.IS on 2023-06-01 00:00:00
Entry Price: 10.529999732971191, Profit Target: 11.612691059442984, Stop Loss: 9.988654069735295

Long signal for YKBNK.IS on 2023-06-02 00:00:00
Entry Price: 11.550000190734863, Profit Target: 12.705356449706509, Stop Loss: 10.972322061249042

Long signal for YKBNK.IS on 2023-06-05 00:00:00
Entry Price: 12.069999694824219, Profit Target: 13.242830479478624, Stop Loss: 11.483584302497016

Long signal for YKBNK.IS on 2023-06-06 00:00:00
Entry Price: 11.949999809265137, Profit Target: 13.084771208561971, Stop Loss:

In [10]:
# Backtest the strategy and calculate performance
def backtest(data, signals):
    balance = 10000  # Starting balance
    position_size = 100  # Fixed position size (number of shares)
    trades = []

    for signal in signals:
        position, entry_date, entry_price, atr = signal
        profit_target, stop_loss = calculate_targets(position, entry_price, atr)

        # Find the price action after the entry signal to determine the outcome
        after_signal = data.loc[entry_date:].copy()

        # Initialize trade outcome variables
        exit_price = entry_price
        trade_outcome = None

        for j in range(1, len(after_signal)):
            current_price = after_signal['Close'].iloc[j]
            date = after_signal.index[j]

            # Check if profit target or stop-loss was hit
            if position == 'Long':
                if current_price >= profit_target:
                    exit_price = profit_target
                    trade_outcome = 'Win'
                    break
                elif current_price <= stop_loss:
                    exit_price = stop_loss
                    trade_outcome = 'Loss'
                    break
            elif position == 'Short':
                if current_price <= profit_target:
                    exit_price = profit_target
                    trade_outcome = 'Win'
                    break
                elif current_price >= stop_loss:
                    exit_price = stop_loss
                    trade_outcome = 'Loss'
                    break

        # If the trade is still active at the end of the data, close at the final price
        if trade_outcome is None:
            exit_price = after_signal['Close'].iloc[-1]
            trade_outcome = 'Closed'

        # Calculate profit or loss
        if position == 'Long':
            profit_or_loss = (exit_price - entry_price) * position_size
        else:
            profit_or_loss = (entry_price - exit_price) * position_size

        # Update balance
        balance += profit_or_loss

        # Record trade details
        trades.append({
            'Ticker': data['Close'].name,
            'Entry Date': entry_date,
            'Position': position,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Profit/Loss': profit_or_loss,
            'Trade Outcome': trade_outcome,
            'Final Balance': balance
        })

    return trades, balance

In [11]:
# Main function to loop through tickers and backtest the strategy
def main():
    total_balance = 10000
    all_trades = []

    for ticker in tickers:
        print(f'Processing {ticker}...')

        # Get stock data
        data = get_stock_data(ticker)

        # Apply the strategy to identify signals
        signals = apply_strategy(data)

        # Backtest the strategy and record trades
        trades, final_balance = backtest(data, signals)
        total_balance = final_balance
        all_trades.extend(trades)

        # Print out trade summary for the ticker
        print(f'Final balance for {ticker}: {final_balance}')
        print(f'Number of trades: {len(trades)}\n')

    # Convert the trades to a DataFrame for analysis
    trades_df = pd.DataFrame(all_trades)

    # Show performance summary
    print(f'Total final balance: {total_balance}')
    print(trades_df[['Ticker', 'Entry Date', 'Position', 'Entry Price', 'Exit Price', 'Profit/Loss', 'Trade Outcome']])

    return trades_df

# Run the main function
if __name__ == '__main__':
    trades_df = main()

Processing AKBNK.IS...


[*********************100%***********************]  1 of 1 completed


Final balance for AKBNK.IS: 19301.097752891765
Number of trades: 245

Processing ALARK.IS...


[*********************100%***********************]  1 of 1 completed


Final balance for ALARK.IS: 35088.04503167895
Number of trades: 235

Processing ASELS.IS...


...

Final balance for TOASO.IS: 59002.956999058304
Number of trades: 245

Processing TUPRS.IS...


[*********************100%***********************]  1 of 1 completed


Final balance for TUPRS.IS: 41883.036472601285
Number of trades: 238

Processing YKBNK.IS...


[*********************100%***********************]  1 of 1 completed


Final balance for YKBNK.IS: 20677.863105291286
Number of trades: 246

Total final balance: 20677.863105291286
     Ticker Entry Date Position  Entry Price  Exit Price  Profit/Loss  \
0     Close 2023-05-29    Short    14.890000   15.809518   -91.951760   
1     Close 2023-05-31    Short    15.800000   16.760299   -96.029839   
2     Close 2023-06-02     Long    17.549999   19.523269   197.326964   
3     Close 2023-06-05     Long    18.170000   20.125179   195.517904   
4     Close 2023-06-06     Long    18.040001   19.918381   187.838034   
...     ...        ...      ...          ...         ...          ...   
7118  Close 2024-09-05     Long    31.940001   30.713631  -122.636916   
7119  Close 2024-09-06     Long    31.260000   30.055514  -120.448579   
7120  Close 2024-09-24     Long    30.860001   31.520000    65.999985   
7121  Close 2024-09-25     Long    31.080000   31.520000    44.000053   
7122  Close 2024-09-26     Long    31.520000   31.520000     0.000000   

     Trade Ou

In [12]:
trades_df

,Ticker,Entry Date,Position,Entry Price,Exit Price,Profit/Loss,Trade Outcome,Final Balance
0,Close,2023-05-29,Short,14.890000,15.809518,-91.951760,Loss,9908.048240
1,Close,2023-05-31,Short,15.800000,16.760299,-96.029839,Loss,9812.018402
2,Close,2023-06-02,Long,17.549999,19.523269,197.326964,Win,10009.345366
3,Close,2023-06-05,Long,18.170000,20.125179,195.517904,Win,10204.863270
4,Close,2023-06-06,Long,18.040001,19.918381,187.838034,Win,10392.701304
...,...,...,...,...,...,...,...,...
7118,Close,2024-09-05,Long,31.940001,30.713631,-122.636916,Loss,20688.311646
7119,Close,2024-09-06,Long,31.260000,30.055514,-120.448579,Loss,20567.863067
7120,Close,2024-09-24,Long,30.860001,31.520000,65.999985,Closed,20633.863052
7121,Close,2024-09-25,Long,31.080000,31.520000,44.000053,Closed,20677.863105


In [ ]:
file_name = 'vegas_backtest.csv'
trades_df.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>